# tensorflow运作方式入门

In [ ]:
#本次的内容主要是讲解部分案例
#数据集还是MNIST
#运行的所有的代码均在第三次的压缩包里，只要运行fully_connected_feed.py即可
#ps.我的会现实没有tf.compact问题，不知道为什么，还希望大家一起来dubug


In [ ]:
#准备数据 数据包使用的是input_data，这个包和第入门2的包是不同的，要放在不同的目录下使用
#这个包的引用的数据是tensorflow自带的MNIST的数据，不是从官网下载的四个gz包
#help：FLAGS怎么用不懂
data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

#输入和占位符
#shape可以控制此占位符的大小，操作与入门2定义的那个NONE，786一样，也可以定义多维变量
images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,IMAGE_PIXELS))
labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
#构建图表
#数据创建占位符后可以运行mnist.py文件 具体的操作查看代码，在此仅讲解大体过程
#inference() 尽可能地构建好图表，满足促使神经网络向前反馈并做出预测的要求
#loss()  往inference图表中添加生成损失（loss）所需要的操作（ops）
#training()  往损失图表中添加计算并应用梯度（gradients）所需的操作



In [ ]:
#推理inference讲解
inference()会构建图表，返回预测结果的tensor
它接受图像占位符为输入，借助Relu(Rectified Linear Units)激活函数，构建一对完整的全连接层，以及一个有着十个节点，指明了输出logtis模型的线性层
每一层都创建一个tf.name_scope
with tf.name_scope('hidden1')as scope:
每一层生成的权重和偏差都在tf.Variable中保存
weithts=Variable(tf.truncated_normal([IMAGE_PIXELSS,hidden1_units],stddev=1.0/math.sqrt(float(IMAGE_PIXELS)))),name=‘weights')
biases=tf.Variable(tf.zeros([hidden1_units])),name='biases')
当这些层是在hidden1作用域下生成时，赋予权重变量的独特名称会是hidden1/weights

hidden1=tf.nn.relu(tf.matmul(inages,weights)+biases)
hidden2=tf.nn.relu(tf.matmul(hidden1,weithts)+biases)
logits=tf.matmul(hidden2,weights)+biases
最后返回了logits 的tensor


In [ ]:
#损失Loss
labels_placeholer 用来存储标识
batch_size=tf.size(labels)
labels=tf.expand_dims(labels,1)
indices=tf.expand_dims(tf.range(0,batch_size,1),1)
concated=tf.concat(1,[indices,labels])
onehot_labels=tf.sparse_to_dense(concated,tf.pack([batch_size,NUM_CLASSES]),1.0,0.0)
这里就是用来计算实际的预测值与本身的数据的差距，得出的loss就是他们的相似度
这里参照入门2的代码比较好理解


In [ ]:
#训练
training() 函数添加了梯度下降的方法，将损失最小化
#首先函数使用loss函数获取损失的tensor，将他提交给tf.scalar_summary
tf.scalar_summary(loss.op.name,loss)
#实例化一个梯度下降法进行训练
optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
#保存训练的数值
global_step = tf.Variable(0, name='global_step', trainable=False)
#更新三角权重
train_op = optimizer.minimize(loss, global_step=global_step)

In [ ]:
#训练模型
图表构建完成后，可以使用full_connected_feed进行循环迭代和评估
之后的循环会话省略


In [ ]:
#检查运行的状态
for step in xrange(FLAGS.max_steps):
    feed_dict = fill_feed_dict(data_sets.train,
                               images_placeholder,
                               labels_placeholder)
    _, loss_value = sess.run([train_op, loss],
                             feed_dict=feed_dict)
    if step % 100 == 0:
    print 'Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration)
    #会返回在运算过程中的过程

In [ ]:
#状态可视化
#即时数据都要放在一个图表中 op操作
summary_op=tf.merge_all_summaries()
#创建好会话之后 使用以下语句将图表本身和即时数据放入具体的事件文件
summary_writer=tf.tarin.SummaryuWriter(FLAGS.train_dir,graph_def=sess.graph_def)
#最后运行summary_op
summary_str = sess.run(summary_op, feed_dict=feed_dict)
summary_writer.add_summary(summary_str, step)
#之后就可以使用训练的文件夹打开tensorBoard查看即时的数据


In [ ]:
#保存检查点
#为了方便中断模型训练后能够继续训练
saver=tf.train.Saver()
#训练中定期调用saver.save()的方法，向训练文件夹中写入包含当前所有可训练变量值的检查点文件
saver.save(sess,FLAGS,train_dir,global_step=step)
#使用的时候使用saver.restore()进行重新加载
saver.restore(sess,FLAGES.train_dir)

In [ ]:
#模型评估
#每过一千个训练步骤，代码会使用训练的数据集和测试集，对模型进行评估
#do_eval函数会被调用三次，分别是训练数据集，验证数据级，和集合测试集
print 'Training Data Eval:'
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.train)
print 'Validation Data Eval:'
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.validation)
print 'Test Data Eval:'
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.test)

In [ ]:
#构建评估图表
#抓取测试数据集
test_all_images,test_all_labels=get_data(train=False)
#调用mnist中的evaluation函数，传入的logits和标签参数要与Loss函数的一致，事前构建eval
eval_correct=mnist.evaluation(logits,labels_placeholder)
#evaluation会生成tf.nn.in_top_k的操作，如果在K个最有可能的预测总发现真的标签，那么这个操作将会输出莫i
#模型标记正确，在本文中我们把K设置为1，也只有在预测是真的标签时，才判断他是正确的
eval_correct=tf.nn.in_top_k(logits,labels,1)


In [ ]:
#评估图标的输出
#创建一个循环，往其中添加feed_dict，并在调用sess.run()函数时传入eval_correct的操作，目的是用给定的数据集评估模型
for step in xrange(steps_per_epoch):
    feed_dict = fill_feed_dict(data_set,
                               images_placeholder,
                               labels_placeholder)
    true_count += sess.run(eval_correct, feed_dict=feed_dict)
#true_count变量会累加所有的in_top_k的操作判定为正确的预测之和，接下来将正确的测试的总数，除以例子的总数，就得出准确率了
precision = float(true_count) / float(num_examples)
print '  Num examples: %d  Num correct: %d  Precision @ 1: %0.02f' % (
    num_examples, true_count, precision)